In [2]:
def generar_clave_ElGamal():
    # Retorna :
    # Genera una clave privada y una clave publica según el protocolo
    # criptográfico ElGamal, para el grupo alamacenado en grupo.txt.
    # Almacena la clave privada en private_key.txt, y la clave publica
    # en public_key.txt
    return 

In [3]:
def firmar_Schnorr(m: str) -> (int, int):
    # Argumentos :
    # m: str - mensaje
    # Retorna :
    # (int, int) - firma de Schnorr (e, s) del mensaje m según la clave
    # privada almacenada en private_key.txt, para el grupo almacenado
    # en grupo.txt
    return

In [4]:
def verificar_firma_Schnorr(m: str, firma: (int, int)) -> bool:
    # Argumentos :
    # m: str - mensaje
    # firma: (int, int) - firma de Schnorr (e, s) para m
    # Retorna: 
    # bool - retorna True si para el usuario con clave publica
    # almacenada en public_key.txt, el par (e, s) es una firma de
    # Schnorr correcta para el mensaje m, en el grupo almacenado en
    # grupo.txt. En caso contrario retorna False
    return

In [5]:
def md5(m: str) -> int:
    # Argumentos :
    # m: str - mensaje
    # Retorna :
    # int - valor de la funcion de hash MD5 aplicada sobre m
    return